# Problem set 1



In [ ]:
pip install xlrd>=1.2.0

In [ ]:
import altair as alt
# import xlrd
import pandas as pd
import numpy as np
from google.colab import data_table

# For google colab only
data_table.enable_dataframe_formatter()

# Load data
hood_code = {
    "manhattan" : 1,
    "bronx" : 2,
    "brooklyn" : 3,
    "queens" : 4,
    "statenisland" : 5
}

bronx_df = pd.read_excel("rolling_sales/rollingsales_bronx.xlsx", header=4)
brooklyn_df = pd.read_excel("rolling_sales/rollingsales_brooklyn.xlsx", header=4)
manhattan_df = pd.read_excel("rolling_sales/rollingsales_manhattan.xlsx", header=4)
queens_df = pd.read_excel("rolling_sales/rollingsales_queens.xlsx", header=4)
statenisland_df = pd.read_excel("rolling_sales/rollingsales_statenisland.xlsx", header=4)

bronx_df = bronx_df.assign(BOROUGH='bronx')
brooklyn_df = brooklyn_df.assign(BOROUGH='brooklyn')
manhattan_df = manhattan_df.assign(BOROUGH='manhattan')
queens_df = queens_df.assign(BOROUGH='queens')
statenisland_df = statenisland_df.assign(BOROUGH='staten island')

# Combine into a single dataframe
nyc_df = pd.concat([bronx_df, brooklyn_df, manhattan_df, 
                    queens_df, statenisland_df])
nyc_df["NEIGHBORHOOD"] = nyc_df["NEIGHBORHOOD"].str.title()
nyc_df["BOROUGH"] = nyc_df["BOROUGH"].str.title()


nyc_df["SALE PRICE"] = nyc_df["SALE PRICE"].astype("float")
nyc_df["NEIGHBORHOOD"] = nyc_df["NEIGHBORHOOD"].astype("category")
nyc_df["BOROUGH"] = nyc_df["BOROUGH"].astype("category")

#  Last 12 month data frame
nyc_one_year_df = nyc_df
nyc_one_year_df = nyc_df[(nyc_df["SALE DATE"] > "2021-02-01") & \
                         (nyc_df["SALE DATE"] < "2022-03-01")]
nyc_one_year_df = nyc_one_year_df.sort_values(by=["SALE DATE"], ascending=True)


In [ ]:
# Question01
# In the past 12 months, what are the most expensive and least expensive
# neighborhoods in NYC measured by average cost of purchasing a home?

# Get the average sale price for each hood
avg_nyc_hood_sales_df = nyc_one_year_df.groupby(["BOROUGH", "NEIGHBORHOOD"])["SALE PRICE"].mean()

# Top 10 highest avg sale prices
avg_nyc_hood_sales_highest_df = avg_nyc_hood_sales_df.nlargest(10).to_frame().reset_index()

# type(avg_nyc_hood_sales_highest_df)
# Last 10 highest avg sale prices
avg_nyc_hood_sales_lowest_df = avg_nyc_hood_sales_df.nsmallest(10).to_frame().reset_index()

# Plot highest
highest_sale_chart = alt.Chart(avg_nyc_hood_sales_highest_df) \
  .mark_bar() \
  .encode(x=alt.Y("NEIGHBORHOOD:N", sort="y"), y=alt.Y("SALE PRICE:Q"), color="BOROUGH:N") \
  .properties(title='Figure 1a') \

# Plot lowest
lowest_sale_chart = alt.Chart(avg_nyc_hood_sales_lowest_df) \
  .mark_bar() \
  .encode(x=alt.Y("NEIGHBORHOOD:N", sort="y"), y="SALE PRICE:Q", color="BOROUGH:N") \
  .properties(title='Figure 1b') 

# Display
sales_charts = highest_sale_chart | lowest_sale_chart
sales_charts.configure_axis(labelFontSize=12, titleFontSize=16) \
            .configure_title(fontSize=16) \
            .configure_legend(titleFontSize=12, labelFontSize=12)


alt.HConcatChart(...)

In [ ]:
# Question02:
# In the last 12 months, what are the most and least active 
# neighborhoods in terms of number of home purchase activities?

# Group by hood and get count 10 most active
avg_nyc_hood_count_largest_df = nyc_one_year_df.groupby(["BOROUGH", "NEIGHBORHOOD"])["SALE PRICE"] \
                                              .count() \
                                              .nlargest(10) \
                                              .to_frame() \
                                              .reset_index()

# avg_nyc_hood_count_lowest_df
highest_sale_chart = alt.Chart(avg_nyc_hood_count_largest_df) \
  .mark_bar() \
  .encode(x=alt.Y("NEIGHBORHOOD:N", sort="y"), 
          y=alt.Y("SALE PRICE:Q", title='Homes Sold Last 12 Months'), 
          color="BOROUGH:N") \
  .properties(title='Figure 2a') 

# Too many hoods with zero sales. 
# Instead, group by borough and get count.
avg_nyc_boro_count_lowest_df = nyc_one_year_df.groupby(["BOROUGH"])["SALE PRICE"] \
                                              .count() \
                                              .to_frame() \
                                              .reset_index()
# Plot
# Base
avg_nyc_bcl_base = alt.Chart(avg_nyc_boro_count_lowest_df) \
  .encode(theta=alt.Theta("SALE PRICE:Q", stack=True), 
          color=alt.Color("BOROUGH:N"))
# Pie
avg_nyc_bcl_pie = avg_nyc_bcl_base.mark_arc(outerRadius=120)
# Text for pie
avg_nyc_bcl_text = avg_nyc_bcl_base.mark_text(radius=143, size=15) \
                                   .encode(text="SALE PRICE:N")
# Combine pie and text
avg_nyc_bcl_chart = avg_nyc_bcl_pie + avg_nyc_bcl_text
avg_nyc_bcl_chart = avg_nyc_bcl_chart.properties(title='Figure 2b')

# Display
sale_chart = highest_sale_chart | avg_nyc_bcl_chart
sale_chart.configure_axis(labelFontSize=12, titleFontSize=16) \
          .configure_title(fontSize=16) \
          .configure_legend(titleFontSize=12, labelFontSize=12)

alt.HConcatChart(...)

In [ ]:
# Question03:
# In the last 12 months, what are the most popular neighborhood for purchasing 
# single family dwellings?  What about two family dwellings, condos, coops?

# Normalize column values
nyc_dwellings_df = nyc_one_year_df.copy()
nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "01 ONE FAMILY DWELLINGS", "BUILDING CLASS CATEGORY"] = "Single Family"
nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "02 TWO FAMILY DWELLINGS", "BUILDING CLASS CATEGORY"] = "Two Family"

nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "13 CONDOS - ELEVATOR APARTMENTS", "BUILDING CLASS CATEGORY"] = "Condo"
nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "15 CONDOS - 2-10 UNIT RESIDENTIAL", "BUILDING CLASS CATEGORY"] = "Condo"
nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "12 CONDOS - WALKUP APARTMENTS", "BUILDING CLASS CATEGORY"] = "Condo"

nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "09 COOPS - WALKUP APARTMENTS", "BUILDING CLASS CATEGORY"] = "Coop"
nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "10 COOPS - ELEVATOR APARTMENTS", "BUILDING CLASS CATEGORY"] = "Coop"
nyc_dwellings_df.loc[nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "17 CONDO COOPS", "BUILDING CLASS CATEGORY"] = "Coop"

# Filter on building types
nyc_dwellings_df = nyc_dwellings_df[(nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "Single Family") | \
                                    (nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "Two Family")    | \
                                    (nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "Condo")         | \
                                    (nyc_dwellings_df["BUILDING CLASS CATEGORY"] == "Coop")          ]

# Get count of building types for each boro and hood.
nyc_dwellings_df = nyc_dwellings_df.groupby(["BOROUGH", "NEIGHBORHOOD", "BUILDING CLASS CATEGORY"])["BUILDING CLASS CATEGORY"] \
                                   .count() \
                                   .nlargest(15) \
                                   .reset_index(name="Count")
# Plot
# Title: Building Types By Neighborhood
alt.Chart(nyc_dwellings_df) \
   .mark_point(size=60) \
   .encode(x=alt.X("Count:Q", title="Total Purchases"),
           y=alt.Y("NEIGHBORHOOD:N", title="Neighborhood", axis=alt.Axis(grid=True)),
           color=alt.Color('BOROUGH:N', title="Borough", scale=alt.Scale(scheme='set1')),
           shape=alt.Shape("BUILDING CLASS CATEGORY:N", title="Building Type")) \
   .properties(title='Figure 3') \
   .configure_axis(labelFontSize=12, titleFontSize=16) \
   .configure_title(fontSize=16) \
   .configure_legend(titleFontSize=12, labelFontSize=12)

alt.Chart(...)

In [ ]:
# Question04:
# In the last 12 month, which month is the busiest month in terms of home 
# purchase activities?

# Add new column for month
nyc_month_sales_df = nyc_one_year_df.copy()

date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-02-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-02-28")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Feb"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-03-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-03-31")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Mar"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-04-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-04-30")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Apr"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-05-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-05-31")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "May"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-06-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-06-30")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Jun"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-07-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-07-31")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Jul"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-08-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-08-31")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Aug"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-09-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-09-30")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Sept"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-10-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-10-31")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Oct"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-11-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-11-30")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Nov"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2021-12-01") & (nyc_month_sales_df["SALE DATE"] <= "2021-12-30")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Dec"
date_mask = (nyc_month_sales_df["SALE DATE"] >= "2022-01-01") & (nyc_month_sales_df["SALE DATE"] <= "2022-01-31")
nyc_month_sales_df.loc[date_mask, "SALE MONTH"] = "Jan"

# Count total sales by month
nyc_month_sales_df = nyc_month_sales_df.groupby(["SALE MONTH"]) \
                                       .agg(count=pd.NamedAgg(column="SALE MONTH", aggfunc="count")) \
                                       .reset_index()
# Plot
# Title: Total Monthly Purchases Feb 21 - Jan 22
month_sort = ["Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec", "Jan"]
monthly_purchases = alt.Chart(nyc_month_sales_df).mark_line(point = True) \
  .encode(
    x=alt.X("SALE MONTH:N", 
              title="Month", 
              sort=month_sort, 
              axis=alt.Axis(grid=True)),
    y=alt.Y("count:Q",title="Total Purchases"),
    color=alt.value("red")) \
  .properties(title='Figure 4') \
  .configure_axis(labelFontSize=12, titleFontSize=16) \
  .configure_title(fontSize=16) \
  .configure_legend(titleFontSize=12, labelFontSize=12)
monthly_purchases

alt.Chart(...)

In [ ]:
# Question05:
# What is the distribution of the age of homes that was purchased in 
# the past 12 month?

nyc_home_age_df = nyc_one_year_df.copy()

# Filter years
nyc_home_age_df = nyc_home_age_df[(nyc_home_age_df["YEAR BUILT"] > int("1890")) & \
                                  (nyc_home_age_df["YEAR BUILT"] <  int("2021"))]
# Get count by year
nyc_home_age_df = nyc_home_age_df.groupby(["YEAR BUILT"]) \
                                 .agg(count=pd.NamedAgg(column="YEAR BUILT", aggfunc="count")) \
                                 .reset_index()
# Type cast to integer
nyc_home_age_df["YEAR BUILT"] = nyc_home_age_df["YEAR BUILT"].astype("int")

# Plot
nyc_home_age_hist = alt.Chart(nyc_home_age_df) \
  .mark_bar() \
  .encode(x=alt.X("YEAR BUILT", bin=alt.Bin(step=10), axis=alt.Axis(format='d')),
          y='count') \
  .properties(title="Figure 5") \
  .configure_axis(labelFontSize=12, titleFontSize=16) \
  .configure_title(fontSize=16) \
  .configure_legend(titleFontSize=12, labelFontSize=12)
nyc_home_age_hist

alt.Chart(...)

In [ ]:
# Question06:
# How does the pandemic impact home purchase activities in New York city? 
# Is the impact the same for homes in different price range? 

# Load annual sales data
five_year_nyc_list = []
dir = "annual_sales/"

for year in ["2017", "2018", "2019", "2020", "2021"]:
  for boro in ["bronx", "brooklyn", "manhattan", "queens", "statenisland"]:
    # xlsx files are formmated different. Handle differences here.
    extension = ".xlsx"
    if year == "2017":
      extension = ".xls"

    header = 4
    if year in ["2020", "2021"]:
      header = 6

    filename = year + "_" + boro + extension
    file_path = dir + filename
    
    print("processing:",file_path)
    # Read excel data
    boro_df = pd.read_excel(file_path, header=header)

    if year in ["2020", "2021"]:
      boro_df = boro_df.drop(index=0)

    # Tidy data
    boro_df.columns = [x.replace("\n", "") for x in boro_df.columns.to_list()]
    boro_df["SALE YEAR"] = int(year)
    boro_df = boro_df.drop(columns=["BOROUGH"])
    boro_df = boro_df.assign(BOROUGH=boro)
    
    # Add to list of dataframes
    five_year_nyc_list.append(boro_df)

# Combine all dataframes into one dataframe
nyc_five_year_df = pd.concat(five_year_nyc_list)
# Tidy column names
nyc_five_year_df["SALE DATE"] = pd.to_datetime(nyc_five_year_df["SALE DATE"], errors="coerce")

# Create bins for house prices
# Bin1: 0 - 499,999
# Bin2: 500,000 - 1,000,000
# Bin3: 1,000,000 - 5,000,000
# Bin4: 5,000,000 - UP

# Create bins by house price
# Bin1: 0 - 499,999
bin_mask = (nyc_five_year_df["SALE PRICE"] >= 0.0) & (nyc_five_year_df["SALE PRICE"] <= 500000)
nyc_five_year_df.loc[bin_mask, "BIN"] = "$0 - $499,999"
# Bin2: 500,000 - 1,000,000
bin_mask = (nyc_five_year_df["SALE PRICE"] >= 500000) & (nyc_five_year_df["SALE PRICE"] <= 1000000)
nyc_five_year_df.loc[bin_mask, "BIN"] = "$500,000 - $1,000,000"
# Bin3: 1,000,000 - 5,000,000
bin_mask = (nyc_five_year_df["SALE PRICE"] >= 1000000) & (nyc_five_year_df["SALE PRICE"] <= 5000000)
nyc_five_year_df.loc[bin_mask, "BIN"] = "$1,000,000 - $5,000,000"
# Bin4: 5,000,000 - UP
bin_mask = (nyc_five_year_df["SALE PRICE"] >= 5000000)
nyc_five_year_df.loc[bin_mask, "BIN"] = "$5,000,000 - UP"

# Count total sales by bin
nyc_five_year_bin_df = nyc_five_year_df.groupby(["BOROUGH", "SALE YEAR","BIN"]) \
                                       .agg(count=pd.NamedAgg(column="BIN", aggfunc="count")) \
                                       .reset_index()

processing: annual_sales/2017_bronx.xls
processing: annual_sales/2017_brooklyn.xls
processing: annual_sales/2017_manhattan.xls
processing: annual_sales/2017_queens.xls
processing: annual_sales/2017_statenisland.xls
processing: annual_sales/2018_bronx.xlsx
processing: annual_sales/2018_brooklyn.xlsx
processing: annual_sales/2018_manhattan.xlsx
processing: annual_sales/2018_queens.xlsx
processing: annual_sales/2018_statenisland.xlsx
processing: annual_sales/2019_bronx.xlsx
processing: annual_sales/2019_brooklyn.xlsx
processing: annual_sales/2019_manhattan.xlsx
processing: annual_sales/2019_queens.xlsx
processing: annual_sales/2019_statenisland.xlsx
processing: annual_sales/2020_bronx.xlsx
processing: annual_sales/2020_brooklyn.xlsx
processing: annual_sales/2020_manhattan.xlsx
processing: annual_sales/2020_queens.xlsx
processing: annual_sales/2020_statenisland.xlsx
processing: annual_sales/2021_bronx.xlsx
processing: annual_sales/2021_brooklyn.xlsx
processing: annual_sales/2021_manhattan.

In [ ]:
# Question06 continued
# Plot
# Title: Homes Purchased With Prices Binned
month_sort = ["0 - $499,999", "$500,000 - $1,000,000", "$1,000,000 - $5,000,000", "$5,000,000 - UP"]
yearly_purchases = alt.Chart(nyc_five_year_bin_df).mark_line(point = True) \
  .encode(
    y = alt.Y("BIN:N", 
              title="Purchase Price", 
              sort=month_sort, 
              axis=alt.Axis(grid=True)),
    x=alt.X("count:Q",title="Total Purchases"),
    color=alt.Color("SALE YEAR:O", legend=alt.Legend(format='d'), scale=alt.Scale(scheme='set1')), \
    facet=alt.Facet('BOROUGH:N', columns=2, header=alt.Header(labelFontSize=16))) \
  .properties(title='Figure 6') \
  .configure_title(fontSize=16, offset=5, orient='top', anchor='middle') \
  .configure_axis(labelFontSize=12, titleFontSize=16) \
  .configure_legend(titleFontSize=12, labelFontSize=12)
yearly_purchases

alt.Chart(...)